In [1]:
!pip install pyautogen


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import os
openAI_API_Key=os.environ['openAI_API_Key']
llm_config = {"model": "gpt-3.5-turbo"}

In [3]:
import autogen

C:\Users\moham\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
C:\Users\moham\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from autogen import ConversableAgent

In [5]:
Design_problem_identifier = ConversableAgent(
    name="Design_problem_identifier",
    system_message='''
    You are an AI assistant for engineering design.
    Your task is getting a design problem from the customer.
    Thus just ask customer kindly and politely to get the design problem
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [6]:
question_maker = ConversableAgent(
    name="question_maker",
    system_message='''
   You should take the role of a very experienced design engineer.
   As a seasoned design engineer, you will be presented with a design issue and your role will be to inquire about it.
   Your objective is to ask a series of questions related to the design problem, to which the customer will respond.
   The questions should be carefully crafted so that the answers will reveal the specific engineering standards that need to be adhered to for the project.
   Keep generating more questions for better results.
   Return 'TERMINATE' when you have gathered all the information.
   ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [7]:
standard_identifier = ConversableAgent(
    name="standard_identifier",
    system_message='''
    As a highly experienced design engineer, you will be presented with a design problem and relevant information.
    you will need to recommend all engineering standards required to be followed in this project.
    Return 'TERMINATE' 
    when you have gathered all the information.
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [15]:
expert_finder = ConversableAgent(
    name="expert_finder",
    system_message='''
I want to give you a title for an engineering design project and specify the engineering standards that need to be followed.
I need you to list all the experts who need to be involved in this project.
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [20]:
Design_resource_calculator = ConversableAgent(
    name="Design_resource_calculator",
    system_message='''
You should take the role of an estimator.
I will give you all the information about an engineering design project and a list of experts available.
Your task will be to provide a rough estimate of the time and funds required to complete this project.
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [8]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

Now, you can craft a series of tasks to facilitate the onboarding process.

In [21]:
chats = [
    {
        "sender": Design_problem_identifier,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to recommend you your required standards for your engineering design project"
            "Please let me know what is your design problem (project title)",
         "summary_method": "reflection_with_llm",
       
        "max_turns": 1,
        "clear_history" : False
    },
    
    {
        "sender": customer_proxy_agent,
        "recipient": question_maker,
        "message": 
                '''Great! I need more information to recommend your required engineering standards.
                 Could you answer these questions (whatever you know!)
                ''',
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : True
    },
    
        {
        "sender": question_maker,
        "recipient": customer_proxy_agent,
        "message": 
                '''Please give me the information here!
                ''',
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    
            {
        "sender": customer_proxy_agent,
        "recipient": standard_identifier,
        "message": 
                '''These are the standards!
                ''',
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    
        {
        "sender": standard_identifier,
        "recipient": expert_finder,
        "message": 
                '''These are the experts should be involved in the project!
                ''',
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    
            {
        "sender": expert_finder,
        "recipient": Design_resource_calculator,
        "message": 
                '''These are the roughly calculated of time and fund we need!
                ''',
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    
  
]

In [22]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Design_problem_identifier (to customer_proxy_agent):

Hello, I'm here to recommend you your required standards for your engineering design projectPlease let me know what is your design problem (project title)

--------------------------------------------------------------------------------
Replying as customer_proxy_agent. Provide feedback to Design_problem_identifier. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Design a table
customer_proxy_agent (to Design_problem_identifier):

Design a table

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
custom

**Print out the summary**

In [23]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

Design a table.


In order to recommend the required engineering standards for designing a table, additional information is needed such as the intended purpose, materials, weight requirements, safety standards, aesthetic considerations, environmental conditions, assembly needs, budget constraints, and any special features required.


Design a heavy outdoor dining table that is 2 meters in height and 2 meters in width.


1. ASTM F1833-96(2015) - Standard Specification for Dining Tables
2. ANSI/BIFMA X5.5 - Desk and Table Products Safety and Performance Standards
3. ISO 4892-2 - Plastics - Methods of Exposure to Laboratory Light Sources 
4. ASTM D7032 - Standard Specification for Establishing Performance Ratings for Wood-Plastic Composite Deck Boards and Guardrail Systems
5. CEN/TC 207 - Environmental management
6. ISO 1006 - Basic Quantities in Kinematics 
7. ANSI/BIFMA X5.1 - General-Purpose Office Chairs - Tests


Experts involved in the project for designing a heavy outdoor dining ta